## Wedge Project - Sam Beighle MSBA Fall 23
Part 1 - Connecting to Google Big Query Using Clean Files

In [ ]:
pip install google-cloud-bigquery google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client


In [ ]:
pip install pandas-gbq

In [ ]:
pip install google-cloud-storage

In [ ]:
import pandas as pd
import os
import csv

import re
import datetime
import io
from zipfile import ZipFile

import pandas as pd
import numpy as np
import pandas_gbq


from google.cloud import bigquery
from google.oauth2 import service_account


### Connect to GBQ

In [ ]:
service_path = r"C:\Users\Sam Beighle\Documents\ADA"
service_file = r'C:\Users\Sam Beighle\Documents\ADA\wedge-project-sam-beighle-100816fecfc7.json' 
gbq_proj_id = 'wedge-project-sam-beighle' 

private_key = os.path.join(service_path, service_file)
credentials = service_account.Credentials.from_service_account_file(private_key)

client = bigquery.Client(credentials = credentials, project=gbq_proj_id)
for item in client.list_datasets() : 
    print(item.full_dataset_id)

### GBQ and extract Zips, and upload to GBQ as tables

In [ ]:
import os
import zipfile
from io import BytesIO
from google.oauth2 import service_account
from google.cloud import bigquery

# Set your Google Cloud project ID
gbq_proj_id = 'wedge-project-sam-beighle'

# Set the path to your service account key JSON file and authenticate
service_path = r"C:\Users\Sam Beighle\Documents\ADA"
service_file = r'C:\Users\Sam Beighle\Documents\ADA\wedge-project-sam-beighle-100816fecfc7.json'

private_key = os.path.join(service_path, service_file)
credentials = service_account.Credentials.from_service_account_file(private_key)

# Create a BigQuery client
client = bigquery.Client(credentials=credentials, project=gbq_proj_id)

# Path to the zip file containing CSVs
zip_file_path = r'C:\Users\Sam Beighle\Documents\ADA\wedge-clean-files.zip'

# Extract the CSVs from the zip file and upload them as tables
with zipfile.ZipFile(zip_file_path, 'r') as zip_file:
    for csv_file_name in zip_file.namelist():
        if csv_file_name.endswith('.csv'):
            # Extract the table name from the CSV file name (excluding the .csv extension)
            table_name = os.path.splitext(os.path.basename(csv_file_name))[0]
            
            # Specify the dataset and table ID
            dataset_id = 'test'  # Replace with your dataset ID
            table_id = table_name  # Use the CSV file name as the table ID
            
            # Create a BigQuery table schema (assuming you have a header row in your CSV)
            schema = []
            
            # Upload the CSV data as a table
            job_config = bigquery.LoadJobConfig(
                schema=schema,
                source_format=bigquery.SourceFormat.CSV,
            )
            
            with zip_file.open(csv_file_name) as csv_file:
                # Create a BytesIO object to read the binary data
                csv_data_stream = BytesIO(csv_file.read())
                
                # Upload the data as a table
                load_job = client.load_table_from_file(
                    csv_data_stream,
                    dataset_id,
                    table_id,
                    job_config=job_config,
                ).result()
            
            print(f"Uploaded CSV '{csv_file_name}' as table '{table_id}' in dataset '{dataset_id}'.")
